# Exercise 7 Solution: Beamforming

## 1. STFT Setup and Loading Microphone Signals
Load the four microphone recordings and compute STFTs.

In [ ]:
import numpy as np
from scipy.io import wavfile
from scipy.signal import stft, istft, get_window

# 1) Parameters
fs, _ = wavfile.read('noisySensor.wav')  # sampling rate
duration = None
# Frame length 128 ms, shift 32 ms
N = int(0.128 * fs)
hop = int(0.032 * fs)

# 2) Windows: sqrt-Hann for analysis; synthesis window scaled by 1/2
win_analysis = np.sqrt(get_window('hann', N, fftbins=True))
win_synthesis = win_analysis / 2

# 3) Load microphone signals (4-channel file)
fs0, data = wavfile.read('noisySensor.wav')  # shape (num_samples, 4)
assert fs0 == fs, "Sampling rates must match"
mic_signals = data.T  # shape (4, num_samples)

# 4) Compute STFT for each microphone: results shape (4, freq_bins, frames)
mic_stfts = []
for i in range(4):
    f, t, Zxx = stft(mic_signals[i], fs, window=win_analysis,
                     nperseg=N, noverlap=N-hop, return_onesided=True)
    mic_stfts.append(Zxx)
mic_stfts = np.stack(mic_stfts, axis=0)  # (4, freq_bins, frames)

print(f"Computed STFTs: {mic_stfts.shape[0]} mics, {mic_stfts.shape[1]} freq bins, {mic_stfts.shape[2]} frames")

**Question:** Why scale the synthesis window by 1/2?

**Answer:** We use sqrt-Hann windows for perfect reconstruction under 75% overlap (hop=N/4). Scaling the synthesis window by 1/2 compensates for the two overlapping analysis windows, ensuring the overlap-add sums to unity and avoids amplitude modulation.

## 2. Delay-and-Sum Beamformer
Implement the classical delay-and-sum beamformer in the STFT domain.

### 2.1 Steering Vector Computation
Compute time delays and steering vector for θ=π/4.

In [ ]:
# 1) Geometry and DOA
c = 340        # speed of sound (m/s)
d = 0.05       # microphone spacing (m)
theta = np.pi/4  # arrival angle

# 2) Compute time delays tau_i for each mic relative to mic0
tau = np.array([i * d * np.cos(theta) / c for i in range(4)])  # (4,)

# 3) Build steering vectors for each frequency bin k
freq_bins = mic_stfts.shape[1]
steering = np.zeros((4, freq_bins), dtype=complex)
for k in range(freq_bins):
    # frequency value
    f_k = k * fs / N
    steering[:, k] = np.exp(-1j * 2 * np.pi * f_k * tau)

print("Steering vector shape:", steering.shape)

**Question:** Why is τᵢ = i·d·cosθ / c correct?

**Answer:** Under the far-field and free-field assumptions, the path difference between mic0 and mic i is i·d·cosθ. Dividing by c gives the time delay for a plane wave arriving at angle θ.

### 2.2 Beamforming and Inverse STFT
Apply delay-and-sum: Ŝ(k,l) = (1/M)·aᴴ(k)·Y(k,l), then synthesize time signal.

In [ ]:
# 1) Stack STFTs: shape (4, freq_bins, frames)
Y = mic_stfts

# 2) Delay-and-sum combining
M = 4
freq_bins, num_frames = Y.shape[1], Y.shape[2]
DS_spec = np.zeros((freq_bins, num_frames), dtype=complex)
for l in range(num_frames):
    DS_spec[:, l] = np.conj(steering[:, :]) * Y[:, :, l]
    # sum across mics and normalize
    DS_spec[:, l] = DS_spec[:, l].sum(axis=0) / M

# 3) Inverse STFT to time domain
_, ds_output = istft(DS_spec, fs, window=win_synthesis,
                     nperseg=N, noverlap=N-hop, input_onesided=True)

# 4) Write output WAV
wavfile.write('ds_output.wav', fs, np.real(ds_output).astype(np.int16))
print("Delay-and-sum output written to ds_output.wav")

**Question:** Why normalize by M?

**Answer:** Dividing by M ensures unity gain for the desired direction, preventing amplitude scaling and maintaining the signal's original power.

### 2.3 Spectrogram Comparison
Plot spectrogram of mic0 and DS output.

In [ ]:
import matplotlib.pyplot as plt

# Spectrogram of mic0
f0, t0, Z0 = stft(mic_signals[0], fs, window=win_analysis,
                  nperseg=N, noverlap=N-hop)
# Spectrogram of DS output
f1, t1, Z1 = stft(ds_output, fs, window=win_analysis,
                  nperseg=N, noverlap=N-hop)

plt.figure(figsize=(10, 6))
plt.subplot(2,1,1)
plt.pcolormesh(t0, f0, 20*np.log10(np.abs(Z0)+1e-12), shading='gouraud')
plt.title('Noisy mic0'); plt.xlabel('Time [s]'); plt.ylabel('Frequency [Hz]')

plt.subplot(2,1,2)
plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Z1)+1e-12), shading='gouraud')
plt.title('Delay-and-Sum Beamformer'); plt.xlabel('Time [s]'); plt.ylabel('Frequency [Hz]')
plt.tight_layout()
plt.show()

**Answer:**
- The DS output spectrogram shows reduced noise floor across frequencies.
- Speech formants remain intact, confirming directional alignment.

## 3. MVDR Beamformer
Implement the Minimum Variance Distortionless Response beamformer.

### 3.1 Noise Covariance and Filter Computation
Estimate noise covariance from the first second (noise-only), then compute h(k).

In [ ]:
from numpy.linalg import solve

# 1) Number of noise-only frames in first second
frames_per_sec = int(fs / hop)
L = frames_per_sec

# 2) Compute noise covariance Phi_V(k) for each k
Phi_V = np.zeros((freq_bins, 4, 4), dtype=complex)
for k in range(freq_bins):
    # accumulate Y[:,k,l] outer products
    accum = np.zeros((4,4), dtype=complex)
    for l in range(L):
        yk = Y[:, k, l][:, None]  # shape (4,1)
        accum += yk @ yk.conj().T
    Phi_V[k] = accum / L

# 3) Compute MVDR weights h(k)
H_mvdr = np.zeros((freq_bins, 4), dtype=complex)
for k in range(freq_bins):
    a_k = steering[:, k]
    # solve Phi_V[k] * x = a_k
    x = solve(Phi_V[k], a_k)
    H_mvdr[k] = x / (a_k.conj().T @ x)

print("Computed MVDR filters H_mvdr with shape:", H_mvdr.shape)

**Question:** Why use solve(Φ_V, a) instead of inv(Φ_V)?

**Answer:** `solve` is numerically more stable and efficient than explicitly computing the inverse, reducing round-off errors.

### 3.2 Apply MVDR and ISTFT
Filter the STFT and reconstruct time-domain signal.

In [ ]:
# 1) Apply MVDR
MVDR_spec = np.zeros((freq_bins, num_frames), dtype=complex)
for l in range(num_frames):
    MVDR_spec[:, l] = np.conj(H_mvdr) * Y[:, :, l]
    MVDR_spec[:, l] = MVDR_spec[:, l].sum(axis=0)

# 2) Inverse STFT
_, mvdr_output = istft(MVDR_spec, fs, window=win_synthesis,
                       nperseg=N, noverlap=N-hop, input_onesided=True)

# 3) Write output WAV
wavfile.write('mvdr_output.wav', fs, np.real(mvdr_output).astype(np.int16))
print("MVDR output written to mvdr_output.wav")

**Answer:**
The MVDR beamformer minimizes noise power while preserving the desired signal in direction θ, resulting in better noise suppression than DS when noise is correlated.

## 4. Performance on Different Noise Fields
Apply both beamformers to isotropic and directional noise recordings.

In [ ]:
# Helper to process any multichannel file
def beamform_and_save(filename):
    # Load signals
    fs_i, data_i = wavfile.read(filename)
    mic_i = data_i.T
    # STFT
    Z = np.stack([stft(mic_i[i], fs_i, window=win_analysis,
                       nperseg=N, noverlap=N-hop)[2] for i in range(4)], axis=0)
    # DS
    DS = np.sum(np.conj(steering[:,:,None]) * Z, axis=0) / M
    # MVDR
    MVDR = np.zeros_like(DS)
    for l in range(Z.shape[2]):
        MVDR[:,l] = np.sum(np.conj(H_mvdr) * Z[:,:,l], axis=0)
    # ISTFT
    _, ds_o = istft(DS, fs_i, window=win_synthesis,
                    nperseg=N, noverlap=N-hop, input_onesided=True)
    _, mvdr_o = istft(MVDR, fs_i, window=win_synthesis,
                      nperseg=N, noverlap=N-hop, input_onesided=True)
    # Save
    base = filename.replace('.wav','')
    wavfile.write(base + '_ds.wav', fs_i, np.real(ds_o).astype(np.int16))
    wavfile.write(base + '_mvdr.wav', fs_i, np.real(mvdr_o).astype(np.int16))
    return ds_o, mvdr_o

# Process isotropic and directional files
iso_ds, iso_mvdr = beamform_and_save('noisyIsotropic.wav')
dir_ds, dir_mvdr = beamform_and_save('noisyDirectional.wav')

print("Processed isotropic and directional noise recordings.")

**Answer:**
- **Uncorrelated (sensor) noise:** DS ≈ MVDR (identical) as noise is spatially white.
- **Isotropic noise:** MVDR outperforms DS by placing nulls and reducing diffuse noise.
- **Directional noise:** MVDR creates a spatial notch towards noise direction; DS cannot null, so residual remains.

### 4.1 Spectrograms Comparison
Show spectrograms of all enhanced signals for visual comparison.

In [ ]:
import matplotlib.pyplot as plt

def plot_spec(sig, title):
    f, t, Z = stft(sig, fs, window=win_analysis, nperseg=N, noverlap=N-hop)
    plt.pcolormesh(t, f, 20*np.log10(np.abs(Z)+1e-12), shading='gouraud')
    plt.title(title); plt.xlabel('Time [s]'); plt.ylabel('Freq [Hz]')

plt.figure(figsize=(12, 10))

titles = ['Sensor DS','Sensor MVDR','Isotropic DS','Isotropic MVDR','Directional DS','Directional MVDR']
sigs = [ds_output, mvdr_output, iso_ds, iso_mvdr, dir_ds, dir_mvdr]

for i, (sig, title) in enumerate(zip(sigs, titles), 1):
    plt.subplot(3,2,i)
    plot_spec(sig, title)

plt.tight_layout()
plt.show()

**Answer:**
- MVDR yields deeper suppression in isotropic and directional noise.
- DS and MVDR produce identical results for sensor noise, confirming theory.
- Spectrograms show clearer formants and lower noise floor with MVDR in correlated-noise cases.